# Prepare Power Flow Data

This notebook presents the process of data preparation for building power flow regimes. The process is separated into stages (see, [DVC config](../dvc.yaml)):
 - "parse" --- extract necessary parameters from the raw data
 - "transform" --- combine and convert data to use in further steps
 - "prepare" --- build final dataset
 - "model" --- build power flow cases

The naming of variables in the final dataset corresponds to the [project convention](../convention.md).

Since the NREL-118 dataset is mostly intended for OPF tasks, it skips a lot of information that is not necessary to solve these kind of tasks. Thus, to append the data with missing info, [JEAS-118 dataset](http://motor.ece.iit.edu/data/JEAS_IEEE118.doc) is used as the primary source of NREL-118. The following sections describe all the stages of data processing and all the decisions made when preparing the final dataset.

In [1]:
import os

from src.data import (
    parse_nrel118_buses,
    prepare_buses,
    parse_nrel118_lines,
    parse_jeas118_lines,
    parse_jeas118_loads,
    parse_jeas118_trafos,
    parse_nrel118_load_ts,
    prepare_branches,
    prepare_loads,
    transform_loads,
    prepare_loads_ts,
)

## Buses

To build a power flow model, the following information about buses is necessary:
- rated voltage level
- if the bus is in service or out of service
- name (optional)
- region (optional)

Let's load and parse bus data of NREL-118 power system:

In [2]:
path_nrel118_buses = os.path.join(
    "..", "data", "raw", "nrel118", "additional-files-mti-118", "Buses.csv"
)
nrel118_buses = parse_nrel118_buses(raw_data=path_nrel118_buses)
nrel118_buses.head(2)

,name,region,load_participation_factor
0,bus_1,r1,0.047169
1,bus_2,r1,0.018496


The NREL-118 dataset contains only names and regions of buses ("load_participation_factor" is for load modelling, see [Section "Loads"](#loads)). To add missing values, it is assumed the following:
- all buses are in service
- rated voltage level of buses 8, 9, 10, 26, 30, 38, 63, 64, 65, 68, 81, 116 equals to 345 kV, the rest of buses has the voltage level of 138 kV. This corresponds to transformers data from JEAS-118 dataset.

Thus, the final bus data look as follows:

In [3]:
buses = prepare_buses(parsed_data=nrel118_buses)
buses.head(2)

,name,region,in_service,v_rated__kv
0,bus_1,r1,True,138
1,bus_2,r1,True,138


## Branches


"Branches" is a common term both for lines and transformers. The following parameters about branches are necessary to build models:
- start / end buses of the branch
- number of parallel branch systems
- resistance
- reactance
- active conductance
- in service or out of service
- maximum power flow (optional)
- transformation ratio if the branch is a transformer (optional)
- name (optional)

Let's load and parse line data of NREL-118 power system:

In [4]:
path_nrel118_lines = os.path.join(
    "..", "data", "raw", "nrel118", "additional-files-mti-118", "Lines.csv"
)
nrel118_lines = parse_nrel118_lines(raw_data=path_nrel118_lines)
nrel118_lines.head(2)

,name,from_bus,to_bus,max_p__mw,x__pu,r__pu
0,branch_1,bus_1,bus_2,600.0,0.0999,0.0303
1,branch_2,bus_1,bus_3,600.0,0.0424,0.0129


Since the information about active conductance and parallel number is skipped in the NREL-118 dataset, let's load it from the JEAS-118 dataset:

In [5]:
path_jeas118_lines = os.path.join("..", "data", "raw", "jeas118", "JEAS_IEEE118.doc")
jeas118_lines = parse_jeas118_lines(raw_data=path_jeas118_lines)
jeas118_lines.head(2)

,name,from_bus,to_bus,parallel,b__pu
0,branch_1,bus_1,bus_2,1,0.02540
1,branch_2,bus_1,bus_3,1,0.01082


In the NREL-118 dataset, transformers are presented as lines without values of transformation ratio. Therefore, these values will be loaded from JEAS-118 dataset:

In [6]:
path_jeas118_trafos = os.path.join("..", "data", "raw", "jeas118", "JEAS_IEEE118.doc")
jeas118_trafos = parse_jeas118_trafos(raw_data=path_jeas118_trafos)
jeas118_trafos.head(2)

,name,from_bus,to_bus,parallel,trafo_ratio
0,trafo_1,bus_8,bus_5,1,0.985
1,trafo_2,bus_26,bus_25,1,0.960


Thus, the final branch data look as follows:

In [7]:
branches = prepare_branches(
    parsed_nrel118_lines=nrel118_lines,
    parsed_jeas118_lines=jeas118_lines,
    parsed_jeas118_trafos=jeas118_trafos,
    prepared_buses=buses,
)
branches.head(2)

,name,from_bus,to_bus,parallel,in_service,r__ohm,x__ohm,b__µs,trafo_ratio,max_i__ka
0,branch_1,bus_1,bus_2,1,True,5.770332,19.024956,133.375341,NaN,2.510219
1,branch_2,bus_1,bus_3,1,True,2.456676,8.074656,56.815795,NaN,2.510219


## Loads

Here is the list of necessary load variables:

- bus where the load is located
- active and reactive power of the load
- if the load is in service
- name (optional)

The information about a part of the regional active load located in each bus is stored in variable "load_participation_factor" in the bus data of the NREL-118 dataset:

In [8]:
nrel118_buses.head(2)

,name,region,load_participation_factor
0,bus_1,r1,0.047169
1,bus_2,r1,0.018496


Active load value of regions is stored in the time-series NREL-118 data:

In [9]:
path_nrel118_load_ts = os.path.join(
    "..", "data", "raw", "nrel118", "Input files", "RT", "Load"
)
nrel118_loads_ts = parse_nrel118_load_ts(raw_data=path_nrel118_load_ts)
nrel118_loads_ts.head(2)

,datetime,r1,r2,r3
0,2024-01-01 00:00:00,5698.083154,1967.417090,2421.205928
1,2024-01-01 01:00:00,5465.729600,1904.444801,2486.384086


To calculate reactive power of loads, let's get the JEAS-118 data:

In [10]:
path_jeas118_load = os.path.join("..", "data", "raw", "jeas118", "JEAS_IEEE118.doc")
jeas118_loads = parse_jeas118_loads(raw_data=path_jeas118_load)
jeas118_loads.head(2)

,bus_name,p__mw,q__mvar
0,bus_1,54.14,8.66
1,bus_2,21.23,9.55


The JEAS-118 load data will help to estimate the power factor of each load and define its reactive power at each moment of time using time-series data of active power of load:

In [11]:
transformed_loads = transform_loads(
    parsed_nrel118_buses=nrel118_buses, parsed_jeas118_loads=jeas118_loads
)
transformed_loads.head(2)

,name,bus_name,region,load_participation_factor,load_power_factor
0,load_1,bus_1,r1,0.047169,0.987447
1,load_2,bus_2,r1,0.018496,0.911978


Thus, it is necessary to prepare two files with load data. The first will contain the load power variation over time, the other will contain basic load information (location, nominal power, etc.). As a nominal value of power, the load at the first moment of the time-series will be used.

In [12]:
loads = prepare_loads(
    transformed_loads=transformed_loads, parsed_nrel118_loads_ts=nrel118_loads_ts
)
loads.head(2)

,name,bus_name,p__mw,q__mvar
0,load_1,bus_1,268.770998,42.991445
1,load_2,bus_2,105.393569,47.409731


In [13]:
loads_ts = prepare_loads_ts(
    transformed_loads=transformed_loads, parsed_nrel118_loads_ts=nrel118_loads_ts
)
loads_ts.head(2)

,datetime,name,in_service,p__mw,q__mvar
0,2024-01-01 00:00:00,load_1,True,268.770998,42.991445
1,2024-01-01 01:00:00,load_1,True,257.811190,41.238362
